# Predict Workout Model Training

Predict Workout Day and distance

In [ ]:
# import loadCorrelatedData function, and pandas
from utils import loadCorrelatedData, pd

from sklearn import preprocessing
import pickle

In [ ]:
data = loadCorrelatedData()
data.head()

### Feature Extraction

In [ ]:
data['day'] = list(map(lambda t : t.day, data['Activity Date']))
data['hour'] = list(map(lambda t : t.hour, data['Activity Date']))
data['dayOfWeek'] = list(map(lambda t : t.dayofweek, data['Activity Date']))
data['isWeekend'] = ((pd.DatetimeIndex(data['Activity Date']).dayofweek) // 5 == 1).astype(int)


# convert all ride types to numeric value for decision making
le_activity = preprocessing.LabelEncoder()
le_activity.fit(data['Activity Type'])
data['rideType'] = le_activity.transform(data['Activity Type'])


# Transform weather conditions data to numeric value for decision making
le = preprocessing.LabelEncoder()
le.fit(data['weatherDesc'])
data['weather'] = le.transform(data['weatherDesc'])

features: **['hour','dayOfWeek','isWeekend','temp','wind','weather']**

In [ ]:
# Select features as list of array
X = data[['hour','dayOfWeek','isWeekend','temp','wind','weather']]
X = X.to_numpy()
X

### Prepare Predict

In [ ]:
# Set Distance values
Y_distance = data['Distance']
Y_distance = Y_distance.to_numpy()

# Set Ride Type Values
Y_rideType = data['rideType']
Y_rideType = Y_rideType.to_numpy()

## Linear Regression for Distance prediction

In [ ]:
# import Linear Regression from sci-kit learn
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle

# select training data and fit final model
model = LinearRegression()
model.fit(X, Y_distance)

# test prediction with a cold Monday weather data
result_distance = model.predict([[8,0,0,10,15,0]])
print("Result distance prediction=%s" % result_distance)

# test prediction with a sunny Sunday weather data
result_distance = model.predict([[6,6,1,26,3,1]])
print("Result distance prediction=%s" % result_distance)

## Logistic Regression for RideType Prediction

In [ ]:
# import Logistic Regression from sci-kit learn
from sklearn.linear_model import LogisticRegression

# select training data and fit final model
model_lr = LogisticRegression(random_state=0).fit(X, Y_rideType)

# test prediction with a clear sunny Sunday weather data
result_ridetype = model_lr.predict([[8,6,1,20,3,0]])
print("Result type prediction=%s" % result_ridetype)

# test prediction with a cold Sunday weather data
result_ridetype = model_lr.predict([[8,6,1,10,12,1]])
print("Result type prediction=%s" % result_ridetype)

In [ ]:
#Ride for the weekend
result_ridetype = model_lr.predict([[8,6,1,8,12,3],[9,7,1,20,17,3],[8,2,0,14,17,3]])
print("Result type prediction=%s" % result_ridetype)

#Ride for Weekend
result = model.predict([[9,6,1,10,12,3],[9,7,1,20,17,3],[8,2,0,14,17,3]])
print(result)

### Export Model as a file

In [ ]:
# Save to file in the model folder
distance_model_file = "../web/model/distance_model_1.pkl"
with open(distance_model_file, 'wb') as file:
   pickle.dump(model, file)
   
ridetype_model_file = "../web/model/ridetype_model_1.pkl"
with open(ridetype_model_file, 'wb') as file:
   pickle.dump(model_lr, file)